In [71]:
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import pearsonr
from sklearn.covariance import GraphicalLassoCV
from networkx.generators.community import gaussian_random_partition_graph
import scipy.stats as stats
from scipy.stats import ttest_ind
import nibabel as nib 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn import config_context

In [72]:
#to do: create a dataframe to store all of the work in 
#cols = [sample_num,gpe,clus_coeff]


#so now I have 3 lists 
# models = [] the trained models 
# gpes = [] the global path effiency from each model graph 
#clustering_coefficients = [] the clustering coefficients from eahc model graph 




#inside the same loop want to organize this info into a dataframe 

In [73]:
#for streamlined code 
#import the csvs instead of generating them 
#then build the graphs and train them on just 10 regions with bootstrap technique 
#regions that I should check out motivated by biology 

In [74]:
#read in pruned csvs with the empty regions dropped 
#when I read it adds a unamed col so just drop it 
progs_d = pd.read_csv('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/ADNI_pruned_prog_csv.csv')
stable_d = pd.read_csv('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/ADNI_pruned_stable_csv.csv')
print(stable_d)

     Unnamed: 0   RID  PROGRESSES    5    6    7    8     9    10    11  ...  \
0             0   173         0.0  150  150  350  352  5258  6796  1169  ...   
1             1   307         0.0  149  154  347  354  5277  6769  1169  ...   
2             5   397         0.0  150  153  350  350  5269  6813  1174  ...   
3             6   417         0.0  150  153  347  355  5258  6757  1172  ...   
4            11   449         0.0  146  152  352  360  5281  6845  1174  ...   
..          ...   ...         ...  ...  ...  ...  ...   ...   ...   ...  ...   
385         539  6627         0.0  149  154  349  355  5272  6797  1175  ...   
386         540  6632         0.0  152  155  350  355  5279  6850  1167  ...   
387         541  6635         0.0  148  153  349  347  5283  6838  1173  ...   
388         542  6652         0.0  151  152  352  354  5274  6808  1174  ...   
389         543  6668         0.0  147  153  351  352  5303  6825  1169  ...   

       133    134   135   136   137   1

In [75]:
#for the correlation matrix need to only have the values so drop the id and labels 
#when I read it adds weird col "unamed" col so just drop it also 
progs_dd = progs_d.drop(["Unnamed: 0","RID","PROGRESSES"], axis =1)
print(progs_dd)
stable_dd = stable_d.drop(["Unnamed: 0","RID","PROGRESSES"], axis =1)

       5    6    7    8     9    10    11    12     13     14  ...    133  \
0    147  153  349  354  5277  6786  1175  1282  32293  32690  ...  11197   
1    149  138  349  354  5261  6758  1171  1283  32213  32643  ...  11205   
2    149  151  348  355  5289  6766  1171  1285  32082  32421  ...  11225   
3    150  152  352  351  5263  6809  1174  1282  32196  32550  ...  11261   
4    151  154  348  356  5251  6789  1173  1283  32000  32318  ...  11184   
..   ...  ...  ...  ...   ...   ...   ...   ...    ...    ...  ...    ...   
149  149  150  349  354  5260  6753  1170  1284  32089  32476  ...  11260   
150  151  154  348  353  5286  6779  1168  1280  32228  32548  ...  11251   
151  150  150  344  357  5266  6771  1173  1284  32146  32482  ...  11225   
152  150  154  350  352  5282  6812  1175  1279  32424  32764  ...  11277   
153  149  153  349  356  5272  6767  1173  1283  32108  32468  ...  11281   

       134   135   136   137   138   139   140   141   142  
0    11009  58

In [76]:
#only keep the 15 regions I want to look at for mini analysis 
# 7: right amygdala , 
#8: left amy 
#21 l hip
# 22 rhip
#31 left thalamus
#32 right thalamus 
#59 left entorhinal area
#60 right entorhinal area 
#71 left inferior temporal gyrus 
#72 right inf temporal gyrus 
# 93 left middle temporal gyrus 
#94 right middle temporal gyrus 
#107 left parahippocampas gyrus 
#135 left superior temporal gyrus
#136 righy superior temporal gyrus 

progs_dd_curated = progs_dd[['7','8','21','22','31','32','59','60','71','72','93','94','107','135','136']]
stable_dd_curated = stable_dd[['7','8','21','22','31','32','59','60','71','72','93','94','107','135','136']]
print(progs_dd_curated)


       7    8    21    22    31    32    59    60     71     72     93     94  \
0    349  354  1256  1409  2934  2797  2179  2178  10598  11250  12654  12547   
1    349  354  1258  1412  2940  2792  2202  2185  10534  11160  12625  12522   
2    348  355  1255  1411  2935  2798  2160  2174  10583  11239  12540  12466   
3    352  351  1260  1424  2941  2799  2158  2151  10573  11180  12603  12470   
4    348  356  1258  1409  2933  2797  2162  2197  10511  11172  12588  12507   
..   ...  ...   ...   ...   ...   ...   ...   ...    ...    ...    ...    ...   
149  349  354  1259  1411  2935  2799  2199  2188  10600  11216  12624  12571   
150  348  353  1259  1415  2940  2799  2208  2186  10629  11250  12629  12524   
151  344  357  1256  1407  2940  2794  2185  2188  10529  11125  12573  12479   
152  350  352  1255  1412  2943  2799  2191  2204  10635  11246  12656  12594   
153  349  356  1258  1411  2938  2802  2185  2177  10585  11155  12559  12439   

      107   135   136  
0  

In [77]:
#from here on out it's with the curated
#convert to float
progs_f = progs_dd_curated.astype(float)
stable_f = stable_dd_curated.astype(float)
print(progs_f.shape)

(154, 15)


In [78]:
#have to do standard scalar normalization 
#it normalizes by column 
#sklearn standard scalar 
#set the output as a df otherwise default is numpy array 
scaler = StandardScaler().set_output(transform="pandas")
scaler_stab = StandardScaler().set_output(transform="pandas")
#fit to your data and apply the transformation to your data 
progs_norm = scaler.fit_transform(progs_f)
stable_norm = scaler_stab.fit_transform(stable_f)
print(progs_norm.shape)
print(progs_norm)

(154, 15)
            7         8        21        22        31        32        59  \
0    0.228218 -0.122008  0.072656 -0.377778 -0.876893 -0.124158 -0.347631   
1    0.228218 -0.122008  0.581251  0.473605  0.839737 -1.617933  1.001011   
2   -0.253228  0.269435 -0.181641  0.189811 -0.590788  0.174597 -1.461727   
3    1.672553 -1.296338  1.089845  3.879138  1.125842  0.473352 -1.579000   
4   -0.253228  0.660878  0.581251 -0.377778 -1.162998 -0.124158 -1.344454   
..        ...       ...       ...       ...       ...       ...       ...   
149  0.228218 -0.122008  0.835548  0.189811 -0.590788  0.473352  0.825101   
150 -0.253228 -0.513452  0.835548  1.324988  0.839737  0.473352  1.352830   
151 -2.179008  1.052322  0.072656 -0.945367  0.839737 -1.020423  0.004188   
152  0.709663 -0.904895 -0.181641  0.473605  1.698052  0.473352  0.356008   
153  0.228218  0.660878  0.581251  0.189811  0.267527  1.369617  0.004188   

           60        71        72        93        94       107  

In [79]:
#old normalization technique 
# #normalize with z score, negative values are fine it means its an std below mean 
# progs_norm = (progs_f - progs_f.mean()) / progs_f.std()
# stable_norm = (stable_f - stable_f.mean()) / stable_f.std()
# # print(progs_norm)

In [80]:
#NEEDS TO BE 154x15 format for model
progs_norm = progs_norm.T #transpose rows and columns
print(progs_norm.shape)

(15, 154)


In [81]:
#bootstrap 
#before build the graph, randomly sampole weight replacement of whole dataset 
#500 patients, sample 500 times with replacement
#then any quanity you are esitmating, will be different, then ypu use these as samples of your quanity and estimate the variance at the end
#unbiased estimate of the variance 
#if I want different type of test 
#computing var, to see if the error bars overlap (whicvch gives you a p value)


#get estimate of the vairnace from bootstrapping 
#t test the variance of the 2 groups 
#for each one of these vars end up with a boot strap and then a boot strapped 


#should read exactly on how bootstrap works, see what is the estimator 
# in sklearn resample 
#run this on the whole dataset (resample from very beginning)

#whole bootstrapping pipeline 

In [82]:
#run bootstrapping (with replacement) before the graphical lasso cv () 
#random_state = None means random number generator is inistalized, produces different set of rnadom samples each time 
#setting random state to 1 so that its reproducable 

#set random generator outside loop 
#seed everytrhing once at begibning and dont touch it 
np.random.seed(1)


num_bootstrap_samples = 10
bootstrap_precision_matrices = []
bootstrap_samples = []
for x in range(num_bootstrap_samples):
    #sample with replacement, randomstate =1 for reporoducibility 
    bootstrap_sample = progs_norm.sample(n=len(progs_norm),replace=True)
    #append resamples df to a list 
    bootstrap_samples.append(bootstrap_sample)


In [83]:
from tqdm import tqdm

In [84]:
#run graphical model on bootstrapped sample 
models = []
for bootstrap_sample in tqdm(bootstrap_samples):
    #if I give it alphas [1,10] it looks like it drops almost all of the connections 
    
    model = GraphicalLassoCV(cv=2,max_iter=50, tol=1e-3)
    model.fit(bootstrap_sample)
    models.append(model) 

  0%|          | 0/10 [00:00<?, ?it/s]/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
 10%|█         | 1/10 [00:29<04:26, 29.60s/it]/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
 20%|██        | 2/10 [01:02<04:10, 31.28s/it]/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:272: RuntimeWarning: invalid value encountered in multiply
  precision_[indices != idx, idx] = -precision_[idx, idx] * coefs
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:273: RuntimeWarning: invalid value encountered in multiply
  precision_[idx, indices != idx] = -precision_[idx, idx] * coefs
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/nu

In [85]:
#list of 500 samples 
#each sample has shape (154,15)


len(models)

10

In [86]:
#compare to make sure they don't look the same 
bootstrap_samples[0]

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,149,150,151,152,153
32,-0.124158,-1.617933,0.174597,0.473352,-0.124158,-0.422913,-0.124158,0.772107,-0.721668,0.174597,...,-0.721668,0.473352,0.174597,0.772107,-1.020423,0.473352,0.473352,-1.020423,0.473352,1.369617
94,0.109211,-0.300996,-1.219859,-1.154226,-0.547120,-1.301900,-1.465983,1.093707,-0.694794,-1.055776,...,-0.973735,-0.563528,-1.810557,0.207661,0.716317,0.503009,-0.268179,-1.006552,0.880400,-1.662882
107,-0.429747,0.113834,1.065103,-0.022061,-1.245120,0.929207,-0.973329,1.336893,1.065103,0.249730,...,0.249730,0.249730,-0.429747,-0.022061,0.793312,-0.429747,-1.381015,0.793312,-0.837434,-0.022061
71,-0.139214,-1.467478,-0.450526,-0.658067,-1.944822,-0.429772,-1.384461,0.628688,0.400393,-0.076952,...,-0.305247,1.334328,-2.048593,1.168295,-0.056198,-0.097706,0.504163,-1.571248,0.628688,-0.409018
72,0.539926,-1.116422,0.337484,-0.748344,-0.895575,-0.693133,-2.202250,-0.104209,0.871196,-0.177824,...,-0.803556,0.889600,-1.558115,0.319080,0.190253,-0.085805,0.539926,-1.760557,0.466311,-1.208441
94,0.109211,-0.300996,-1.219859,-1.154226,-0.547120,-1.301900,-1.465983,1.093707,-0.694794,-1.055776,...,-0.973735,-0.563528,-1.810557,0.207661,0.716317,0.503009,-0.268179,-1.006552,0.880400,-1.662882
32,-0.124158,-1.617933,0.174597,0.473352,-0.124158,-0.422913,-0.124158,0.772107,-0.721668,0.174597,...,-0.721668,0.473352,0.174597,0.772107,-1.020423,0.473352,0.473352,-1.020423,0.473352,1.369617
7,0.228218,0.228218,-0.253228,1.672553,-0.253228,-0.734673,-0.253228,0.228218,0.228218,-3.623344,...,0.709663,-0.253228,0.709663,1.672553,0.228218,0.228218,-0.253228,-2.179008,0.709663,0.228218
7,0.228218,0.228218,-0.253228,1.672553,-0.253228,-0.734673,-0.253228,0.228218,0.228218,-3.623344,...,0.709663,-0.253228,0.709663,1.672553,0.228218,0.228218,-0.253228,-2.179008,0.709663,0.228218
8,-0.122008,-0.122008,0.269435,-1.296338,0.660878,0.269435,-0.513452,0.269435,-0.513452,-2.862112,...,-0.122008,0.660878,-0.904895,0.269435,-0.513452,-0.122008,-0.513452,1.052322,-0.904895,0.660878


In [87]:
bootstrap_samples[1]

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,149,150,151,152,153
72,0.539926,-1.116422,0.337484,-0.748344,-0.895575,-0.693133,-2.202250,-0.104209,0.871196,-0.177824,...,-0.803556,0.889600,-1.558115,0.319080,0.190253,-0.085805,0.539926,-1.760557,0.466311,-1.208441
21,0.072656,0.581251,-0.181641,1.089845,0.581251,-2.724613,0.072656,0.581251,0.326954,-0.944532,...,0.835548,-0.435938,-0.690235,-0.181641,0.072656,0.835548,0.835548,0.072656,-0.181641,0.581251
31,-0.876893,0.839737,-0.590788,1.125842,-1.162998,0.267527,0.839737,-1.162998,-0.018578,-0.876893,...,-0.876893,-0.018578,-1.449103,0.553632,1.125842,-0.590788,0.839737,0.839737,1.698052,0.267527
136,0.663479,-0.310715,-0.826465,-0.826465,-0.711854,-1.628743,-0.625896,-0.568590,0.090424,-0.052840,...,-0.196104,-1.141646,-0.711854,-0.597243,-0.511285,0.348299,-0.511285,2.927049,-0.081493,-1.256257
32,-0.124158,-1.617933,0.174597,0.473352,-0.124158,-0.422913,-0.124158,0.772107,-0.721668,0.174597,...,-0.721668,0.473352,0.174597,0.772107,-1.020423,0.473352,0.473352,-1.020423,0.473352,1.369617
21,0.072656,0.581251,-0.181641,1.089845,0.581251,-2.724613,0.072656,0.581251,0.326954,-0.944532,...,0.835548,-0.435938,-0.690235,-0.181641,0.072656,0.835548,0.835548,0.072656,-0.181641,0.581251
31,-0.876893,0.839737,-0.590788,1.125842,-1.162998,0.267527,0.839737,-1.162998,-0.018578,-0.876893,...,-0.876893,-0.018578,-1.449103,0.553632,1.125842,-0.590788,0.839737,0.839737,1.698052,0.267527
94,0.109211,-0.300996,-1.219859,-1.154226,-0.547120,-1.301900,-1.465983,1.093707,-0.694794,-1.055776,...,-0.973735,-0.563528,-1.810557,0.207661,0.716317,0.503009,-0.268179,-1.006552,0.880400,-1.662882
107,-0.429747,0.113834,1.065103,-0.022061,-1.245120,0.929207,-0.973329,1.336893,1.065103,0.249730,...,0.249730,0.249730,-0.429747,-0.022061,0.793312,-0.429747,-1.381015,0.793312,-0.837434,-0.022061
93,0.217884,-0.217689,-1.494367,-0.548123,-0.773419,-1.824801,-1.539426,0.863733,-0.473024,0.097726,...,0.127765,-0.382906,-1.899900,0.277963,0.172825,-0.232708,-0.157610,-0.998715,0.247923,-1.208992


In [88]:
#checks the number of unique values in the bootstrap sample 
bootstrap_samples[1].index.nunique()

9

In [89]:
#to do: create a dataframe to store all of the work in 
#cols = [sample_num,gpe,clus_coeff]


#so now I have 3 lists 
# models = [] the trained models 
# gpes = [] the global path effiency from each model graph 
#clustering_coefficients = [] the clustering coefficients from eahc model graph 




#inside the same loop want to organize this info into a dataframe 

In [90]:
#different metrics to calc 
#gpe, clus_coeff 
#density nx.density(G)
#eigenvector centrality nx.eigenvector_centrality(G) seems like its not a single value 
#assostivity coeff nx.degree_assortativity_coefficient(G) 
#diameter nx.diameter(G) -  Gives the diameter of the largest connected component in the graph, representing the longest shortest path between any pair of nodes. gave error 
#radius nx.radius - computes the radius of the largest connected compmenet gave error 
#number connected components nx.number_connected_components(G)


In [109]:
#this is i
# terating through each bootsrap sample model, calculating the graph metrics, and then storing the metrics for eahc model in a dataframe 
metrics_data = []

#keep track of index with enumerate 
for model_num, model in enumerate(models,start=1):
    precision_matrix_p = model.precision_
    np.fill_diagonal(precision_matrix_p, 0) #removes self connections by replacing the diagnonal of matrix with 0 
    G_prog = nx.Graph(precision_matrix_p)
    node_labels = {i: label for i, label in enumerate(progs_norm.columns)}
    #relabel the nodes in the graph using the dictionary
    G_prog = nx.relabel_nodes(G_prog, node_labels)
    #now calculate the metrics and store them 
    gpe = nx.global_efficiency(G_prog) 
    clustering_coefficient = nx.average_clustering(G_prog)
    density = nx.density(G_prog)
    # eigen_cen = nx.eigenvector_centrality(G_prog)
    ass_coeff = nx.degree_assortativity_coefficient(G_prog)
    # diameter = nx.diameter(G_prog)
    # radius = nx.radius(G_prog)


    #create dictionary for current models data 
    metrics_dic = {
        'model_num': model_num,
        'gpe': gpe,
        'clus_coeff':clustering_coefficient,
        'density': density,
        'ass_coeff': ass_coeff
    }
    metrics_data.append(metrics_dic)


#turn the list of dictionaries with each model info into a df
metrics_data = pd.DataFrame(metrics_data)
print(metrics_data.dtypes)
print(metrics_data)

model_num       int64
gpe           float64
clus_coeff    float64
density       float64
ass_coeff     float64
dtype: object
   model_num       gpe  clus_coeff   density  ass_coeff
0          1  0.209112    0.256990  0.024446  -0.390641
1          2  0.144012    0.158454  0.014175  -0.450740
2          3  0.120846    0.228995  0.016807  -0.460532
3          4  0.188811    0.216022  0.026653  -0.443489
4          5  0.297046    0.276792  0.036415  -0.426474
5          6  0.062508    0.087297  0.007809  -0.537048
6          7  0.102529    0.142329  0.013496  -0.467179
7          8  0.177916    0.259095  0.026059  -0.413708
8          9  0.171634    0.185228  0.019099  -0.503395
9         10  0.199305    0.304756  0.024786  -0.518127


In [92]:
#now I have 100 graphs, what are the metrics plot historgram of each metric 
#node specific across all the graphs 


#precision mat per bootstrap sample 
#now comptuing stat for each booptstrap sample 
#computing avg dgeree for each of 100 bootwstrap samples, so 
#Any property of the graph is now 100 samples 
#compute some global graph stat and then compare the mean of that stat/metric across all the samples and compare between groups 
#compare means from 2 groups, calc var and error 

In [93]:
#run bootstrap for stable df 
np.random.seed(1)
stab_num_bootstrap_samples = 10
stab_bootstrap_precision_matrices = []
stab_bootstrap_samples = []
for x in range(stab_num_bootstrap_samples):
    #sample with replacement, randomstate =1 for reporoducibility 
    stab_bootstrap_sample = stable_norm.sample(n=len(stable_norm),replace=True)
    #append resamples df to a list 
    stab_bootstrap_samples.append(stab_bootstrap_sample)

In [94]:
#run graphical model on bootstrapped sample 
stab_models = []
for stab_bootstrap_sample in tqdm(stab_bootstrap_samples):
    #if I give it alphas [1,10] it looks like it drops almost all of the connections 
    
    stab_model = GraphicalLassoCV(cv=2,max_iter=50,tol=1e-3)
    stab_model.fit(stab_bootstrap_sample)
    stab_models.append(stab_model) 

100%|██████████| 10/10 [00:00<00:00, 11.95it/s]


In [95]:
#check that the bootstrap values are unique 
stab_bootstrap_samples[1].index.nunique()

256

In [110]:

# Assuming stable_norm and stab_models are already defined somewhere

stab_metrics_data = []

for model_num, stab_model in enumerate(stab_models):
    precision_matrix_stab = stab_model.precision_
    np.fill_diagonal(precision_matrix_stab, 0)  # removes self connections by replacing the diagonal of matrix with 0
    G_stab = nx.Graph(precision_matrix_stab)
    # create a dictionary that maps old node labels to new node labels
    stab_node_labels = {i: label for i, label in enumerate(stable_norm.columns)}
    # relabel the nodes in the graph using the dictionary
    G_stab = nx.relabel_nodes(G_stab, stab_node_labels)
    # now calculate the metrics and store them
    stab_gpe = nx.global_efficiency(G_stab)
    stab_clustering_coefficient = nx.average_clustering(G_stab)

    # Assuming you want to store metrics for stab (stability) models,
    # not prog (progression) models as shown in the second snippet
    stab_density = nx.density(G_stab)
    stab_ass_coeff = nx.degree_assortativity_coefficient(G_stab)

    # metrics dictionary
    stab_metrics_dic = {'model_num': model_num,
                        'gpe': stab_gpe,
                        'clus_coeff': stab_clustering_coefficient,
                        'density': stab_density,
                        'ass_coeff': stab_ass_coeff
                        }

    stab_metrics_data.append(stab_metrics_dic)

stab_metrics_data = pd.DataFrame(stab_metrics_data)
print(stab_metrics_data)


   model_num       gpe  clus_coeff   density  ass_coeff
0          0  0.741270    0.535820  0.485714  -0.237093
1          1  0.804762    0.577407  0.609524   0.014770
2          2  0.776190    0.511558  0.552381  -0.116486
3          3  0.776190    0.564536  0.552381  -0.256545
4          4  0.828571    0.635960  0.657143  -0.110788
5          5  0.755556    0.575926  0.514286  -0.062901
6          6  0.711111    0.373439  0.438095  -0.071209
7          7  0.823810    0.597465  0.647619  -0.160793
8          8  0.842857    0.708342  0.685714  -0.163712
9          9  0.828571    0.639245  0.657143  -0.192136


In [112]:
#instead of calcing mean just calc from data instead 
#add col where the p val is greater than .05
import pandas as pd
from scipy import stats

# Load your data if not already loaded
# adni_gmv_progs = pd.read_csv('path_to_adni_gmv_progs.csv')
# adni_gmv_stab = pd.read_csv('path_to_adni_gmv_stab.csv')

# Initialize lists or a DataFrame to store your results
cols = []
p_values = []
t_stats = []

# Note: The dataframe should be adni_gmv_progs, not adni_gmv_prog
for col in metrics_data.columns:  
    if col != 'model_num':
    # Get data for this brain region from both datasets
        progs_data = metrics_data[col]
        stab_data = stab_metrics_data[col]
        
        # Perform t-test
        t_stat, p_val = stats.ttest_ind(progs_data, stab_data, equal_var=False)  # Assuming variance might not be equal
        
        # Store results
        cols.append(col)
        t_stats.append(t_stat)
        p_values.append(p_val)

    # Combine results into a DataFrame for easy viewing and further analysis
    results_df = pd.DataFrame({
        'Brain Region': cols,
        'T-Statistic': t_stats,
        'P-Value': p_values
})

# Adding the Significant column based on the P-Value
results_df['Significant'] = results_df['P-Value'] < 0.05

# Display or save the results
print(results_df)
# Optionally save to a CSV file
# results_df.to_csv('t_test_results.csv', index=False)

#the resulting negative t stats mean the progs have lower volume than the stab which is good 


  Brain Region  T-Statistic       P-Value  Significant
0          gpe   -25.059318  3.947791e-14         True
1   clus_coeff   -10.140090  1.454011e-08         True
2      density   -20.961538  4.598972e-09         True
3    ass_coeff   -10.765614  3.349200e-08         True


In [ ]:
#calc the means for prog 
#dont need this because not comparing means 
#somehow the datatype is list not string 
gpes_mean = metrics_df['gpe'].mean()
gpes_std = metrics_df['gpe'].std()
print(f"prog gpes mean {gpes_mean}")
print(f"prog gpes std {gpes_std}")
clustering_coefficients_mean = metrics_df['clus_coeff'].mean()
clustering_coefficients_std = metrics_df['clus_coeff'].std()
print(f"prog cc mean {clustering_coefficients_mean}")
print(f"prog cc std {clustering_coefficients_std}")

#stab means 
stab_gpes_mean = np.mean(stab_gpes)
stab_gpes_std = np.std(stab_gpes)
print(f"stab gpes mean {stab_gpes_mean}")
print(f"stab gpes std {stab_gpes_std}")
stab_clustering_coefficients_mean = np.mean(stab_clustering_coefficients)
stab_clustering_coefficients_std = np.std(stab_clustering_coefficients)
print(f"stab cc mean {stab_clustering_coefficients_mean}")
print(f"stab cc std {stab_clustering_coefficients_std}")




prog gpes mean 0.7961809523809524
prog gpes std 0.04627393621291616
prog cc mean 0.5882318178118178
prog cc std 0.0913304093809654
stab gpes mean 0.8462126984126985
stab gpes std 0.040075420330781784
stab cc mean 0.6903541443741444
stab cc std 0.07916141540641318


In [ ]:
#now that I have this, just scale it to the whole brains and all patients, and more metrics 

teststat,pvalue = ttest_ind(gpes_mean,stab_gpes_mean)
print(teststat)
print(pvalue)

teststat,pvalue = ttest_ind(clustering_coefficients_mean,stab_clustering_coefficients_mean)
print(teststat)
print(pvalue)

NameError: name 'ttest_ind' is not defined

In [ ]:
#from this code onwards it's a duplicate of the old stuff 